## Task 1

In [42]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score


In [43]:
# Create the data for the Blue population
blue_data = {
    "Will use XAI": [60, 20],
    "Will not use XAI": [5, 15],
    "Total": [65, 35]
}

# Create dataframe for Blue population
df_blue = pd.DataFrame(blue_data, index=["Enrolled in training", "Not enrolled in training"])

total_red = 100 / 9
total_blue = 100

# Create the data for the Red population
# Red group has half enrolled in training, so split evenly
red_enrolled = total_red / 2
red_not_enrolled = total_red / 2

red_data = {
    "Will use XAI": [red_enrolled / 2, red_not_enrolled / 2],
    "Will not use XAI": [red_enrolled / 2, red_not_enrolled / 2],
    "Total": [red_enrolled, red_not_enrolled]
}

# Create dataframe for Red population
df_red = pd.DataFrame(red_data, index=["Enrolled in training", "Not enrolled in training"])



In [44]:
# Fairness metric calculations

# 1. Demographic Parity
# Formula: (Proportion enrolled from Blue group) / (Proportion enrolled from Red group)

demographic_parity = (df_blue.loc["Enrolled in training", "Total"] / df_blue["Total"].sum()) / \
                     (df_red.loc["Enrolled in training", "Total"] / df_red["Total"].sum())

# 2. Equal Opportunity
# Formula: (True positive rate for Blue group) / (True positive rate for Red group)
equal_opportunity = (df_blue.loc["Enrolled in training", "Will use XAI"] / df_blue["Will use XAI"].sum()) / \
                    (df_red.loc["Enrolled in training", "Will use XAI"] / df_red["Will use XAI"].sum())

# 3. Predictive Rate Parity
# Formula: (Precision for Blue group) / (Precision for Red group)
# Precision is True Positive / (True Positive + False Positive)
precision_blue = df_blue.loc["Enrolled in training", "Will use XAI"] / df_blue.loc["Enrolled in training", "Total"]
precision_red = df_red.loc["Enrolled in training", "Will use XAI"] / df_red.loc["Enrolled in training", "Total"]

predictive_rate_parity = precision_blue / precision_red

demographic_parity, equal_opportunity, predictive_rate_parity


(1.3, 1.5, 1.8461538461538463)

## Task 2

In [45]:
data = pd.read_csv('compas-scores-two-years.csv', delimiter=",", index_col='id')

data

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,0,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,0,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,0,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0


In [46]:
important_columns = [
    'age', 'age_cat', 'sex', 'v_score_text', 'priors_count', 'decile_score',
    'juv_fel_count', 'juv_misd_count', 'c_charge_degree', 'is_recid', 'two_year_recid'
]

protected = pd.get_dummies(data['race'], columns=['race'], drop_first=False)


filtered_data = data[important_columns].copy()

categorical_columns = ['age_cat', 'sex', 'v_score_text', 'c_charge_degree']

filtered_data_encoded = pd.get_dummies(filtered_data, columns=categorical_columns, drop_first=True)
filtered_data_encoded = filtered_data_encoded.astype(float)

filtered_data_encoded

,age,priors_count,decile_score,juv_fel_count,juv_misd_count,is_recid,two_year_recid,age_cat_Greater than 45,age_cat_Less than 25,sex_Male,v_score_text_Low,v_score_text_Medium,c_charge_degree_M
id,,,,,,,,,,,,,
1,69.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,34.0,0.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,24.0,4.0,4.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
5,23.0,1.0,8.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
6,43.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10996,23.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
10997,23.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
10999,57.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [47]:
filtered_data_encoded['protected'] = protected['African-American'].astype(float)


In [48]:
# Define features and target variable
X = filtered_data_encoded.drop(columns=['two_year_recid'])
y = filtered_data_encoded['two_year_recid']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

protected = X_test['protected']

X_train = X_train.drop(columns=['protected'])
X_test = X_test.drop(columns=['protected'])


### Logistic Regression

In [49]:
# Train a logistic regression classifier
model_linear = LogisticRegression(max_iter=1000, random_state=42)
model_linear.fit(X_train, y_train)

y_pred_lin = model_linear.predict(X_test)

y_pred_lin

array([0., 0., 1., ..., 1., 0., 1.])

### Decision Tree

In [50]:
model_tree = tree.DecisionTreeClassifier()

model_tree = model_tree.fit(X_train, y_train)

y_pred_tree = model_tree.predict(X_test)

y_pred_tree

array([0., 0., 1., ..., 0., 0., 1.])

In [51]:
# Calculate statistics for the tree model
accuracy = accuracy_score(y_test, y_pred_tree)
precision = precision_score(y_test, y_pred_tree, zero_division=1)
recall = recall_score(y_test, y_pred_tree, zero_division=1)
f1 = f1_score(y_test, y_pred_tree, zero_division=1)

print(f'Tree model: accuracy = {accuracy}, precision = {precision}, recall = {recall}, F1 = {f1}')

# Calculate statistics for the logistic regression model
accuracy = accuracy_score(y_test, y_pred_lin)
precision = precision_score(y_test, y_pred_lin, zero_division=1)
recall = recall_score(y_test, y_pred_lin, zero_division=1)
f1 = f1_score(y_test, y_pred_lin, zero_division=1)

print(f'Linear model: accuracy = {accuracy}, precision = {precision}, recall = {recall}, F1 = {f1}')



Tree model: accuracy = 0.9441108545034642, precision = 0.9504843918191603, recall = 0.9217118997912317, F1 = 0.9358770535241123
Linear model: accuracy = 0.9750577367205543, precision = 0.9466403162055336, recall = 1.0, F1 = 0.9725888324873097


In [52]:
protected_group = (protected == 1)
non_protected_group = (protected == 0)

def calculate_fairness(preds):
    # 1. Demographic Parity
    demographic_parity = np.mean(preds[protected_group]) / np.mean(preds[non_protected_group])

    # 2. Equal Opportunity
    true_positive_protected = np.mean((preds == 1) & (y_test.to_numpy() == 1) & (protected == 1))
    true_positive_non_protected = np.mean((preds == 1) & (y_test.to_numpy() == 1) & (protected == 0))
    equal_opportunity = true_positive_protected / true_positive_non_protected

    # 3. Predictive Parity
    precision_protected = true_positive_protected / np.mean(preds[protected_group])
    precision_non_protected = true_positive_non_protected / np.mean(preds[non_protected_group])
    predictive_parity = precision_protected / precision_non_protected

    return demographic_parity, equal_opportunity, predictive_parity

d_lin, e_lin, p_lin = calculate_fairness(y_pred_lin)
d_tree, e_tree, p_tree = calculate_fairness(y_pred_tree)
print(f'Fairness for linear model: \n Demographic Parity: {d_lin}, \n Equal Opportunity: {e_lin}, \n Predictive Parity {p_lin} \n')
print(f'Fairness for tree model: \n Demographic Parity: {d_tree}, \n Equal Opportunity: {e_tree}, \n Predictive Parity {p_tree} \n')



Fairness for linear model: 
 Demographic Parity: 1.4428474676617808, 
 Equal Opportunity: 1.5343915343915342, 
 Predictive Parity 1.0634468083297162 

Fairness for tree model: 
 Demographic Parity: 1.5035019338289815, 
 Equal Opportunity: 1.589442815249267, 
 Predictive Parity 1.057160472817896 



In [58]:
probs_lin = model_linear.predict_proba(X_test)[:, 1]

# Bonuses to test
bonuses = [0.3, 0.35, 0.4, 0.45]

results = []

for bonus in bonuses:
    probs_lin_mit = probs_lin.copy()
    
    # Apply the mitigation by subtracting the bonus for the protected group
    probs_lin_mit[protected_group] = probs_lin_mit[protected_group] - bonus
    
    # Threshold the probabilities to get predictions
    y_pred_lin_mit = np.zeros_like(probs_lin_mit)
    y_pred_lin_mit[probs_lin_mit > 0.5] = 1

    # Calculate fairness metrics for the mitigated model
    demographic_parity, equal_opportunity, predictive_parity = calculate_fairness(y_pred_lin_mit)
    
    # Store the results
    results.append({
        'Bonus': bonus,
        'Demographic Parity': demographic_parity,
        'Equal Opportunity': equal_opportunity,
        'Predictive Parity': predictive_parity
    })

fairness_df = pd.DataFrame(results)

print(fairness_df)


   Bonus  Demographic Parity  Equal Opportunity  Predictive Parity
0   0.30            1.442847           1.534392           1.063447
1   0.35            1.442847           1.534392           1.063447
2   0.40            1.263377           1.351852           1.070030
3   0.45            0.491182           0.529101           1.077198


In [59]:
# Calculate statistics for the logistic regression model
accuracy = accuracy_score(y_test, y_pred_lin_mit)
precision = precision_score(y_test, y_pred_lin_mit, zero_division=1)
recall = recall_score(y_test, y_pred_lin_mit, zero_division=1)
f1 = f1_score(y_test, y_pred_lin_mit, zero_division=1)

# Output the statistics
print(f'Linear model mittigated: accuracy = {accuracy}, precision = {precision}, recall = {recall}, F1 = {f1}')

Linear model mittigated: accuracy = 0.810161662817552, precision = 0.9490968801313628, recall = 0.6033402922755741, F1 = 0.7377153797064454
